# Soft Body Simulation

# Point Mass Class

In [ ]:
class point_mass:
    def __init__(self, mass, x, y):
        self.mass = mass
        self.x = x
        self.y = y
        self.vx = 0
        self.vy = 0
    
    def apply_F(self, Fx, Fy): 
        self.vy += Fy / self.mass
        self.vx += Fx / self.mass

    def update_pos(self):
        self.x += self.vx
        self.y += self.vy
    

### Spring Class

#### <center>Hooke's Law defines the Spring Force as: </center>
#### <center>F = -k * x</center>

In [ ]:
class spring:
    def __init__(self, pm_1, pm_2, k):
        self.pm_structure = [pm_1, pm_2]
        self.k = k
        self.length = ((abs(pm_1.x - pm_2.x))**2 + (abs(pm_1.y - pm_2.y))**2)**0.5
        self.x_length = (pm_1.x - pm_2.x)
        self.y_length = (pm_1.y - pm_2.y)
        self.Fx_spring = 0
        self.Fy_spring = 0
    
    def find_Fspring(self):
        stretched_x_length = self.pm_1.x - self.pm_2.x
        stretched_y_length = self.pm_1.y - self.pm_2.y
        delta_x = self.x_length - stretched_x_length
        delta_y = self.y_length - stretched_y_length
        self.Fx_spring = -self.k * (delta_x) # negative when spring is stretched (force points inwards); positive when spring compressed (force points outwards)
        self.Fy_spring = -self.k * (delta_y)

    def apply_Fspring(self):
        self.find_Fspring()
        self.pm_1.apply_F(self.Fx_spring, self.Fy_spring)
        self.pm_2.apply_F(self.Fx_spring, self.Fy_spring)


# Shapes

In [ ]:
class Shape:
    def __init__(self, x, y, mass, k):
        self.x = x
        self.y = y
        self.mass = mass
        self.k = k
        self.pm_structure = []  
        self.spring_structure = []  

    def apply_Fgravity(self):
        for pm in self.pm_structure:
            Fx = 0
            Fy = pm.mass * -9.81 # assuming we calculate Fy at each time set, t = 1
            pm.apply_F(Fx, Fy)
    

    def check_springs(self):
        for spring in self.spring_structure:
            spring.find_Fspring()

    def update_pos(self):
        for pm in self.pm_structure:
            pm.update_pos()
    

In [18]:
class Square(Shape):
    def __init__(self, x, y, size, mass, k):
        super().__init__(x, y, mass, k)
        self.size = size
        self.pm_structure = [point_mass((mass/4), (x-size), (y+size)), # top left square corner
                                point_mass((mass/4), (x+size), (y+size)), # top right square corner
                                point_mass((mass/4), (x+size), (y-size)), # bottom right square corner
                                point_mass((mass/4), (x-size), (y-size)) # bottom left square corner
                                ]
        self.spring_structure = [spring(self.pm_structure[0], self.pm_structure[1], self.k), # spring connecting top left, top right
                                 spring(self.pm_structure[1], self.pm_structure[2], self.k), # spring connecting top right, bottom right
                                 spring(self.pm_structure[2],self. pm_structure[3], self.k), # spring connecting bottom right, bottom left
                                 spring(self.pm_structure[3], self.pm_structure[0], self.k)] # spring connecting bottom left, top left

class Triangle(Shape):
    def __init__(self, x, y, size, mass, k):
        super().__init__(x, y, mass, k)
        self.size = size
        self.pm_structure = [point_mass((mass/3), (x), (y + size)), # top point
                                point_mass((mass/3), (x-(size/2)), (y)), # bottom left point
                                point_mass((mass/3), (x+(size/2)), (y)) # bottom right point
                            ]
        self.spring_structure = [spring(self.pm_structure[0], self.pm_structure[1], self.k), # spring connecting top point, bottom left
                                 spring(self.pm_structure[1], self.pm_structure[2], self.k), # spring connecting bottom left, bottom right
                                 spring(self.pm_structure[2],self. pm_structure[0], self.k)] # spring connecting bottom right, top point



# Main